### Super Store Dataset Analysis & Visualisation


<p aligh='justify'>This notebook proivdes descriptive analysis of the Super Store dataset. To reproduce data structure in database follow instructions in "Config/ProjectSetup.md". Dataset includes fully detailed information about orders in SuperStore.</p>

Information provided in dataset: 
* Row ID - unique identifier of the record 
* Order ID - identifier of particular order 
* Order Date - purchase order timestamp 
* Ship Date - delivery timestamp 
* Ship Mode - picked delivery option 
* Customer ID - unique identifier of the customer 
* Customer Name - name & and surnmae of customer 
* Segment - customer's segment e.g. customer classification 
* Country - customer country (data only for US)
* City - customer city 
* State - customer state 
* Postal Code - unique identifier of the customer localization 
* Region - particular region of the US
* Product ID - product identifier (unfortunately not unique)
* Category - product main category 
* Sub-Category - additional category of the product 
* Sales - sum of sales for order 
* Quantity - amount of the product 
* Discount - discount rate 
* Profit - total profit from order 


<p align='justify'>For analysis purpose column Country is removed from database - it has not brought any value or information. Additionaly unique record identifier and unique order identifier has been derived.  </p>

Dependency resolving and sql extension laod.

In [1]:
%pip install -r config/requirements.txt

^C
Note: you may need to restart the kernel to use updated packages.


In [2]:
%load_ext sql

In [24]:
%sql mssql+pyodbc://PLLUKOR1@local

In [4]:
import numpy as np
import pandas as pd

Data sample

In [5]:
%%sql 
use SuperStore;

SELECT  TOP 5
        A.OderDate
       ,A.ShipDate
       ,B.ShipModeName
       ,C.CustomerID
       ,C.CustomerName
       ,D.SegmentName
       ,F.CityName
       ,G.StateName
       ,H.RegionName
       ,E.PostalCode
       ,I.ProductID
       ,K.CategoryName
       ,J.SubCategoryName
       ,I.ProductName
       ,A.Sales
       ,A.Quantity
       ,A.Discount
       ,A.Profit
FROM Orders A
LEFT JOIN ShipModels B
ON A.ShipModeID = B.ShipModeID
LEFT JOIN Customers C
ON A.CustomerID = C.CustomerID
LEFT JOIN Segments D
ON C.SegmentID = D.SegmentID
LEFT JOIN Localisations E
ON C.PostalCode = E.PostalCode
LEFT JOIN Cities F
ON E.CityID = F.CityID
LEFT JOIN States G
ON E.StateID = G.StateID
LEFT JOIN Regions H
ON E.RegionID = H.RegionID
LEFT JOIN Products I
ON A.UniqueProductID = I.UniqueProductID
LEFT JOIN SubCategories J
ON I.SubCategoryID = J.SubCategoryID
LEFT JOIN Categories K
ON J.CategoryID = K.CategoryID

 * mssql+pyodbc://PLLUKOR1@local
Done.
Done.


OderDate,ShipDate,ShipModeName,CustomerID,CustomerName,SegmentName,CityName,StateName,RegionName,PostalCode,ProductID,CategoryName,SubCategoryName,ProductName,Sales,Quantity,Discount,Profit
2016-08-11 00:00:00,2016-11-11 00:00:00,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,42420,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96,2,0.00,41.91
2016-08-11 00:00:00,2016-11-11 00:00:00,Second Class,CG-12520,Claire Gute,Consumer,Henderson,Kentucky,South,42420,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs, Rounded Back",731.94,3,0.00,219.58
2016-12-06 00:00:00,2016-06-16 00:00:00,Second Class,DV-13045,Darrin Van Huff,Corporate,Los Angeles,California,West,90036,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters by Universal,14.62,2,0.00,6.87
2015-11-10 00:00:00,2015-10-18 00:00:00,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,33311,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.58,5,0.45,-383.03
2015-11-10 00:00:00,2015-10-18 00:00:00,Standard Class,SO-20335,Sean O'Donnell,Consumer,Fort Lauderdale,Florida,South,33311,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.37,2,0.20,2.52


TOP 10 States by Sales

In [6]:
query = '''
SELECT  TOP 10 C.StateName
       ,SUM(A.Sales) AS 'Sum'
       ,AVG(A.Sales) AS Average
       ,MIN(A.Sales) AS Minimum
       ,MAX(A.Sales) AS Maximum
FROM Orders A
JOIN Localisations B
ON A.PostalCode = B.PostalCode
JOIN States C
ON B.StateID = C.StateID
GROUP BY  C.StateName
ORDER BY SUM(A.Sales) desc
'''
result = %sql $query
result = pd.DataFrame(result)
result


 * mssql+pyodbc://PLLUKOR1@local
Done.


,StateName,Sum,Average,Minimum,Maximum
0,California,450041.53,229.378965,0.99,8187.65
1,New York,310876.20,275.599468,1.24,11199.97
2,Texas,170187.98,172.779675,0.44,8159.95
3,Washington,138641.29,273.994644,1.34,13999.96
4,Pennsylvania,116512.02,198.487257,0.85,8399.98
5,Florida,89473.73,233.612872,1.17,22638.48
6,Illinois,80166.16,162.939349,0.84,2799.96
7,Ohio,78258.21,166.861855,1.45,4499.99
8,Michigan,76269.61,299.096509,2.20,9892.74
9,Virginia,70636.72,315.342500,2.84,8749.95


Regions by Sales

In [7]:
query = '''
SELECT  C.RegionName
       ,SUM(A.Sales) AS 'Sum'
       ,AVG(A.Sales) AS Average
       ,MIN(A.Sales) AS Minimum
       ,MAX(A.Sales) AS Maximum
FROM Orders A
JOIN Localisations B
ON A.PostalCode = B.PostalCode
JOIN Regions C
ON B.RegionID = C.RegionID
GROUP BY  RegionName
'''

result = %sql $query
result = pd.DataFrame(result)
result

 * mssql+pyodbc://PLLUKOR1@local
Done.


,RegionName,Sum,Average,Minimum,Maximum
0,East,678781.36,238.336151,0.85,11199.97
1,South,391721.90,241.803641,1.17,22638.48
2,West,717811.78,226.868451,0.99,13999.96
3,Central,501239.88,215.772656,0.44,17499.95


TOP 10 Customers

In [8]:
query = '''
SELECT  TOP 10 B.CustomerName 
       ,SUM(A.Sales) AS 'Sum'
       ,AVG(A.Sales) AS Average
       ,MIN(A.Sales) AS Minimum
       ,MAX(A.Sales) AS Maximum
FROM Orders A
JOIN Customers B 
ON A.CustomerID = B.CustomerID
GROUP BY  B.CustomerName 
ORDER BY SUM(A.Sales) desc
'''

result = %sql $query
result = pd.DataFrame(result)
result


 * mssql+pyodbc://PLLUKOR1@local
Done.


,CustomerName,Sum,Average,Minimum,Maximum
0,Sean Miller,25043.07,1669.538000,3.49,22638.48
1,Tamara Chand,19052.22,1587.685000,7.31,17499.95
2,Raymond Buch,15117.35,839.852777,4.45,13999.96
3,Tom Ashbrook,14595.62,1459.562000,7.04,11199.97
4,Adrian Barton,14473.57,723.678500,3.69,9892.74
5,Ken Lonsdale,14175.23,488.801034,1.19,8187.65
6,Sanjit Chand,14142.34,642.833636,5.04,9449.95
7,Hunter Lopez,12873.30,1170.300000,6.63,10499.97
8,Sanjit Engle,12209.44,642.602105,5.95,8749.95
9,Christopher Conant,12129.08,1102.643636,1.25,8399.98


Most Frequent Products

In [9]:
query = '''
SELECT  top 10 B.ProductName
       ,COUNT(A.Sales) AS TotalSales
FROM Orders A
JOIN Products B
ON A.UniqueProductID = B.UniqueProductID
GROUP BY  B.ProductName
ORDER BY TotalSales desc;
'''

result = %sql $query
result = pd.DataFrame(result)
result

 * mssql+pyodbc://PLLUKOR1@local
Done.


,ProductName,TotalSales
0,Staple envelope,48
1,Staples,46
2,Easy-staple paper,46
3,Avery Non-Stick Binders,20
4,Staples in misc. colors,19
5,Xerox 1881,19
6,Imation 16GB Mini TravelDrive USB 2.0 Flash Drive,18
7,KI Adjustable-Height Table,18
8,Staple remover,18
9,Storex Dura Pro Binders,17


Most profitable products

In [10]:
query = '''
SELECT  top 10 B.ProductName
       ,COUNT(A.Sales) AS TotalSold
       ,SUM(A.Sales)   AS TotalSales
FROM Orders A
JOIN Products B
ON A.UniqueProductID = B.UniqueProductID
GROUP BY  B.ProductName
ORDER BY TotalSales desc;
'''

result = %sql $query
result = pd.DataFrame(result)
result

 * mssql+pyodbc://PLLUKOR1@local
Done.


,ProductName,TotalSold,TotalSales
0,Canon imageCLASS 2200 Advanced Copier,5,61599.83
1,Fellowes PB500 Electric Punch Plastic Comb Bin...,10,27453.38
2,Cisco TelePresence System EX90 Videoconferenci...,1,22638.48
3,HON 5400 Series Task Chairs for Big and Tall,8,21870.57
4,GBC DocuBind TL300 Electric Binding System,11,19823.48
5,GBC Ibimaster 500 Manual ProClick Binding System,9,19024.50
6,Hewlett Packard LaserJet 3310 Copier,8,18839.68
7,HP Designjet T520 Inkjet Large Format Printer ...,3,18374.90
8,GBC DocuBind P400 Electric Binding System,6,17965.07
9,High Speed Automatic Electric Letter Opener,3,17030.31


Sales & Profit By Year

In [23]:
query = '''
SELECT  YEAR(OderDate) AS 'Year'
       ,COUNT(Sales)   AS TotalSold
       ,SUM(Sales)     AS TotalSales
       ,SUM(Profit)    AS TotalProfit
       , CONCAT(CAST(SUM(Profit)/SUM(Sales)*100 
              AS DECIMAL(10,2)), '%') as ProfitRatio
FROM Orders
GROUP BY  YEAR(OderDate)
ORDER BY Year asc;
'''

result = %sql $query
result = pd.DataFrame(result)
result

 * mssql+pyodbc://PLLUKOR1@local
Done.


,Year,TotalSold,TotalSales,TotalProfit,ProfitRatio
0,2014,1980,480113.28,48690.21,10.14%
1,2015,2096,469726.96,61404.95,13.07%
2,2016,2578,607472.25,81686.60,13.45%
3,2017,3301,732242.43,93154.68,12.72%


Montly trends

In [26]:
query = '''
SELECT MONTH(OderDate) AS 'Month'
       ,COUNT(Sales)    AS TotalSold
       ,SUM(Sales)      AS TotalSales
       ,SUM(Profit)     AS TotalProfit
       , CONCAT(CAST(SUM(Profit)/SUM(Sales)*100 
              AS DECIMAL(10,2)), '%') as ProfitRatio
FROM Orders
GROUP BY  MONTH(OderDate)
ORDER BY Month asc;
'''

result = %sql $query
result = pd.DataFrame(result)
result

 * mssql+pyodbc://PLLUKOR1@local
Done.


,Month,TotalSold,TotalSales,TotalProfit,ProfitRatio
0,1,597,161083.61,25167.08,15.62%
1,2,548,132721.37,23753.71,17.90%
2,3,818,219898.28,29256.68,13.30%
3,4,693,146874.32,12209.90,8.31%
4,5,823,166021.89,24182.79,14.57%
5,6,704,144724.22,21117.72,14.59%
6,7,737,160935.10,9960.75,6.19%
7,8,810,207537.56,24146.53,11.63%
8,9,1137,248980.77,34551.21,13.88%
9,10,807,184320.05,22324.87,12.11%
